In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import os
from typing import List
from tqdm import tqdm
import fire
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoModelForCausalLM,  BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
)
from fed_utils import FedAvg, client_selection, global_evaluation, GeneralClient
import datasets
from utils.prompter import Prompter
datasets.utils.logging.set_verbosity_error()

In [3]:
ME="/dpc/kunf0007/amine"

In [4]:
mylora = "amew0/mylora-shepherd"
loraCache = f"{ME}/tokenizer"

myTokenizer = LlamaTokenizer.from_pretrained(mylora,cache_dir=loraCache,padding_side="right",pad_token_id=(0))

In [5]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    mylora,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"": 0},
    cache_dir=f"{ME}/model"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

In [6]:
# Encode some text
text = "What are all the positions in hockey?"
input_ids = myTokenizer.encode(text, return_tensors="pt")

In [7]:
# Generate predictions
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

/dpc/kunf0007/amine/.conda/envs/llama2/lib/python3.10/site-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [9]:
# Decode the generated tokens to get the text
predicted_text = myTokenizer.decode(output[0], skip_special_tokens=True)
print(predicted_text)

What are all the positions in hockey?
The answer is no, not all the positions. The most common positions in hockey are the five “F” positions: Forward, Defense, Goaltender, Center, and Wing. There


In [13]:
chavinlo = "chavinlo/alpaca-native"
chavinlo_tokenizer = LlamaTokenizer.from_pretrained(
    chavinlo,
    cache_dir=f"{ME}/chavinlo/tokenizer",
    padding_side="right",
    pad_token_id=(0))

In [14]:
chavinlo_model = AutoModelForCausalLM.from_pretrained(
    chavinlo,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"": 0},
    cache_dir=f"{ME}/chavinlo/model"
)

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

In [ ]:
input_ids = chavinlo_tokenizer.encode(text, return_tensors="pt")
output = chavinlo_model.generate(input_ids, max_length=50, num_return_sequences=1)
predicted_text = chavinlo_tokenizer.decode(output[0], skip_special_tokens=True)
print(predicted_text)